In [23]:
pip install google-api-python-client


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install textblob


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install python-dotenv


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install seaborn


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
# src/clean_data.py
import pandas as pd
import os
import json
from datetime import datetime

# --- Safe directory handling (works in VS Code + Jupyter) ---
BASE_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()

RAW_DATA_DIR = os.path.join(BASE_DIR, '../data/raw')
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, '../data/processed')
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)


def load_raw_data():
    """
    Load the most recent raw JSON files.
    Your get_data.py saves files as:
        artist_summary_YYYYMMDD.json
        content_detail_YYYYMMDD.json
    So we match that pattern.
    """

    artist_files = sorted(
        [f for f in os.listdir(RAW_DATA_DIR) if f.startswith("artist_summary_")],
        reverse=True
    )
    content_files = sorted(
        [f for f in os.listdir(RAW_DATA_DIR) if f.startswith("content_detail_")],
        reverse=True
    )

    if not artist_files or not content_files:
        print("No raw JSON files found. Run get_data.py first.")
        return pd.DataFrame(), pd.DataFrame()

    # Load most recent pair
    artist_path = os.path.join(RAW_DATA_DIR, artist_files[0])
    content_path = os.path.join(RAW_DATA_DIR, content_files[0])

    print(f"Loading artist summary → {artist_path}")
    with open(artist_path, "r") as f:
        artists_raw = json.load(f)

    print(f"Loading content detail → {content_path}")
    with open(content_path, "r") as f:
        content_raw = json.load(f)

    df_artists = pd.DataFrame(artists_raw)
    df_content = pd.DataFrame(content_raw)

    return df_artists, df_content


def clean_and_process_data(df_art_raw, df_content_raw):
    """
    CLEANING RULES FOR YOUR PROJECT:
    - No Spotify data (removed)
    - Artists have:
        tiktok_views, tiktok_video_count,
        subscriber_count, view_count, video_count
    - Content has YouTube videos with:
        view_count, like_count, comment_count,
        sentiment ratios, published_at
    """

    print("Cleaning data...")

    # --- Clean artist-level summary ---
    df_art = df_art_raw.copy()

    # Convert numeric cols if exist
    for col in ["tiktok_views", "tiktok_video_count",
                "subscriber_count", "view_count", "video_count"]:
        if col in df_art.columns:
            df_art[col] = pd.to_numeric(df_art[col], errors="coerce").fillna(0)

    # timestamp formatting
    if "timestamp" in df_art.columns:
        df_art["timestamp"] = pd.to_datetime(df_art["timestamp"], errors="coerce")

    # Drop garbage rows (shouldn't happen)
    if "artist_name" in df_art.columns:
        df_art.dropna(subset=["artist_name"], inplace=True)

    # --- Clean content-level data (YouTube videos) ---
    df_con = df_content_raw.copy()

    # Parse timestamps
    if "timestamp" in df_con.columns:
        df_con["timestamp"] = pd.to_datetime(df_con["timestamp"], errors="coerce")
    if "published_at" in df_con.columns:
        df_con["published_at"] = pd.to_datetime(df_con["published_at"], errors="coerce")

    # Make numeric
    for col in ["view_count", "like_count", "comment_count"]:
        if col in df_con.columns:
            df_con[col] = pd.to_numeric(df_con[col], errors="coerce").fillna(0)

    # Engagement rate
    if all(c in df_con.columns for c in ["like_count", "comment_count", "view_count"]):
        df_con["engagement_rate"] = (
            (df_con["like_count"] + df_con["comment_count"]) /
            df_con["view_count"].replace(0, 1)
        )
    else:
        df_con["engagement_rate"] = 0

    # Sentiment fallback
    for col in ["sentiment_positive_ratio", "sentiment_negative_ratio", "sentiment_neutral_ratio"]:
        if col in df_con.columns:
            df_con[col] = df_con[col].fillna(0)
        else:
            df_con[col] = 0

    # Clean artist name
    if "artist_name" in df_con.columns:
        df_con.dropna(subset=["artist_name"], inplace=True)

    print("Cleaning complete.")
    return df_art, df_con


def save_processed_data(df_art, df_con):
    art_path = os.path.join(PROCESSED_DATA_DIR, "cleaned_artists_data.csv")
    con_path = os.path.join(PROCESSED_DATA_DIR, "cleaned_content_data.csv")

    df_art.to_csv(art_path, index=False)
    df_con.to_csv(con_path, index=False)

    print(f"Saved cleaned artists → {art_path}")
    print(f"Saved cleaned content → {con_path}")


if __name__ == "__main__":
    df_art_raw, df_con_raw = load_raw_data()
    if df_art_raw.empty or df_con_raw.empty:
        print("No raw data to clean.")
    else:
        df_art, df_con = clean_and_process_data(df_art_raw, df_con_raw)
        save_processed_data(df_art, df_con)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/henryfuckingdopeman/Desktop/DSCI-510/../data/raw'

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from googleapiclient.discovery import build
import pandas as pd
import time
import json
from datetime import datetime
from textblob import TextBlob
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# --- YouTube API ---
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")

# Output dir
#RAW_DATA_DIR = os.path.join(os.path.dirname(__file__), '../data/raw')
#os.makedirs(RAW_DATA_DIR, exist_ok=True)
BASE_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
RAW_DATA_DIR = os.path.join(BASE_DIR, '../data/raw')

# Initialize YouTube
youtube = None
if YOUTUBE_API_KEY:
    try:
        youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)
        print("YouTube API client initialized.")
    except Exception as e:
        print("YouTube init error:", e)
else:
    print("YouTube API key missing — skipping YouTube collection.")


# -------------------------------------------
#  🔥 TikTok Scraping
# -------------------------------------------
def scrape_tiktok_hashtag_stats(hashtag):
    url = f"https://www.tiktok.com/tag/{hashtag}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        for script in soup.find_all("script"):
            if "props" in script.text:
                txt = script.text

                if '"viewCount":' in txt:
                    v1 = txt.index('"viewCount":') + len('"viewCount":')
                    v2 = txt.index(",", v1)
                    views = int(txt[v1:v2].strip())

                    if '"videoCount":' in txt:
                        c1 = txt.index('"videoCount":') + len('"videoCount":')
                        c2 = txt.index(",", c1)
                        vids = int(txt[c1:c2].strip())

                        return {
                            "hashtag": hashtag,
                            "tiktok_views": views,
                            "tiktok_video_count": vids
                        }
        return None
    except Exception as e:
        print(f"TikTok scrape failed for #{hashtag}: {e}")
        return None


# -------------------------------------------
#  🔥 YouTube Stats
# -------------------------------------------
def get_youtube_channel_stats(channel_id):
    if not youtube:
        return None
    try:
        req = youtube.channels().list(part="statistics,snippet", id=channel_id)
        res = req.execute()
        if not res["items"]:
            return None

        stats = res["items"][0]["statistics"]
        snip = res["items"][0]["snippet"]

        return {
            "channel_id": channel_id,
            "channel_name": snip["title"],
            "subscriber_count": int(stats.get("subscriberCount", 0)),
            "view_count": int(stats.get("viewCount", 0)),
            "video_count": int(stats.get("videoCount", 0)),
        }
    except:
        return None


def get_youtube_video_stats_and_comments(channel_id, max_videos=10, max_comments=10):
    if not youtube:
        return []

    videos = []
    try:
        search = youtube.search().list(
            part="id,snippet", channelId=channel_id,
            maxResults=max_videos, order="date", type="video"
        )
        res = search.execute()
        ids = [i["id"]["videoId"] for i in res["items"]]

        if not ids:
            return []

        stats_req = youtube.videos().list(part="statistics,snippet", id=",".join(ids))
        stats_res = stats_req.execute()

        for item in stats_res["items"]:
            vid = {
                "video_id": item["id"],
                "video_title": item["snippet"]["title"],
                "published_at": item["snippet"]["publishedAt"],
                "view_count": int(item["statistics"].get("viewCount", 0)),
                "like_count": int(item["statistics"].get("likeCount", 0)),
                "comment_count": int(item["statistics"].get("commentCount", 0)),
            }

            # Sentiment
            comments = get_youtube_comments_for_sentiment(item["id"], max_comments)
            pos = neg = neu = 0

            for c in comments:
                p = TextBlob(c).sentiment.polarity
                if p > 0.1: pos += 1
                elif p < -0.1: neg += 1
                else: neu += 1

            total = len(comments) or 1
            vid["sentiment_positive_ratio"] = pos / total
            vid["sentiment_negative_ratio"] = neg / total
            vid["sentiment_neutral_ratio"] = neu / total

            videos.append(vid)
        return videos

    except:
        return []


def get_youtube_comments_for_sentiment(video_id, max_comments=10):
    comments = []
    try:
        req = youtube.commentThreads().list(
            part="snippet", videoId=video_id,
            maxResults=max_comments, textFormat="plainText"
        )
        res = req.execute()

        for item in res.get("items", []):
            comments.append(
                item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            )
    except:
        pass

    return comments


# -------------------------------------------
# 🎤 Artist List (10 artists)
# -------------------------------------------
ARTISTS_TO_ANALYZE = [
    {"name": "Taylor Swift", "tiktok_tag": "taylorswift", "youtube_channel_id": "UCqfmriSjJ_k4C8W6J_k7J_g"},
    {"name": "NBA YoungBoy", "tiktok_tag": "nbayoungboy", "youtube_channel_id": "UCNofc_JcK-0FfdJ_YkE6VBg"},
    {"name": "Adele", "tiktok_tag": "adele", "youtube_channel_id": "UCRw-9o3C02JkL4o1CjDkywA"},
    {"name": "Bad Bunny", "tiktok_tag": "badbunny", "youtube_channel_id": "UCgCHiixL-q7L5_Fv2EaV3-w"},
    {"name": "Billie Eilish", "tiktok_tag": "billieeilish", "youtube_channel_id": "UCiGm_E4ZwYVaeYBjfK6edYA"},
    {"name": "Drake", "tiktok_tag": "drake", "youtube_channel_id": "UCByOQJjav0CUDwxCk-jVNRQ"},
    {"name": "The Weeknd", "tiktok_tag": "theweeknd", "youtube_channel_id": "UC0WP5P-ufpRfjbNrmOWwLBQ"},
    {"name": "Doja Cat", "tiktok_tag": "dojacat", "youtube_channel_id": "UCzvK5p4gGg9Q2HfKF3Ab4Qw"},
    {"name": "Post Malone", "tiktok_tag": "postmalone", "youtube_channel_id": "UC3gK4uQkzkG4gQ2vRGLcH3A"},
    {"name": "Kendrick Lamar", "tiktok_tag": "kendricklamar", "youtube_channel_id": "UC3lBXkFNkSgWunf6Z64MfKQ"}
]


# -------------------------------------------
# 🔥 Main collection
# -------------------------------------------
def collect_and_save_data(artists_list, timestamp):
    summary = []
    detail = []

    for a in artists_list:
        print(f"\nCollecting data for {a['name']}...")

        # TikTok
        tiktok = scrape_tiktok_hashtag_stats(a["tiktok_tag"])
        if tiktok:
            tiktok.update({
                "artist_name": a["name"],
                "data_type": "tiktok_stats",
                "timestamp": timestamp.isoformat(),
            })
            summary.append(tiktok)

        # YouTube channel
        yt = get_youtube_channel_stats(a["youtube_channel_id"])
        if yt:
            yt.update({
                "artist_name": a["name"],
                "data_type": "youtube_channel_stats",
                "timestamp": timestamp.isoformat(),
            })
            summary.append(yt)

        # YouTube videos
        vids = get_youtube_video_stats_and_comments(a["youtube_channel_id"])
        for v in vids:
            v.update({
                "artist_name": a["name"],
                "data_type": "youtube_video",
                "timestamp": timestamp.isoformat(),
            })
            detail.append(v)

        time.sleep(1)

    ts = timestamp.strftime("%Y%m%d_%H%M%S")
    with open(os.path.join(RAW_DATA_DIR, f"artist_summary_{ts}.json"), "w") as f:
        json.dump(summary, f, indent=4)

    with open(os.path.join(RAW_DATA_DIR, f"content_detail_{ts}.json"), "w") as f:
        json.dump(detail, f, indent=4)

    print("\n✔ Data saved successfully.")
    return summary, detail


# -------------------------------------------
# Run script
# -------------------------------------------
if __name__ == "__main__":
    now = datetime.now()
    collect_and_save_data(ARTISTS_TO_ANALYZE, now)


YouTube API key missing — skipping YouTube collection.












FileNotFoundError: [Errno 2] No such file or directory: '/../data/raw/artist_summary_20251210_152657.json'

In [ ]:
# src/run_analysis.py
import os
import pandas as pd
import numpy as np
from datetime import datetime

# --- Directory handling that works in VS Code + Jupyter ---
BASE_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, '../data/processed')
RESULTS_DIR = os.path.join(BASE_DIR, '../results')
os.makedirs(RESULTS_DIR, exist_ok=True)

CLEANED_ARTISTS_CSV = os.path.join(PROCESSED_DATA_DIR, 'cleaned_artists_data.csv')
CLEANED_CONTENT_CSV = os.path.join(PROCESSED_DATA_DIR, 'cleaned_content_data.csv')


# ----------------------
# Helper utilities
# ----------------------
def first_existing_column(df, candidates, default=None):
    """Return first column name in candidates that exists in df."""
    for c in candidates:
        if c in df.columns:
            return c
    return default


def safe_to_datetime(series):
    """Convert to datetime without errors."""
    return pd.to_datetime(series, errors='coerce')


def manual_minmax(series):
    """Min-max normalize using Python math instead of sklearn."""
    series = pd.to_numeric(series, errors='coerce').fillna(0)
    mn, mx = series.min(), series.max()
    if mx - mn == 0:
        return pd.Series([0] * len(series))
    return (series - mn) / (mx - mn)


# ----------------------
# Load cleaned data
# ----------------------
def load_cleaned_data():
    if not os.path.exists(CLEANED_ARTISTS_CSV) or not os.path.exists(CLEANED_CONTENT_CSV):
        print("❌ Cleaned data missing. Run clean_data.py first.")
        return pd.DataFrame(), pd.DataFrame()

    df_artists = pd.read_csv(CLEANED_ARTISTS_CSV)
    df_content = pd.read_csv(CLEANED_CONTENT_CSV)

    # Fix timestamps
    for col in ['collection_timestamp', 'published_at', 'release_date']:
        if col in df_artists.columns:
            df_artists[col] = safe_to_datetime(df_artists[col])
        if col in df_content.columns:
            df_content[col] = safe_to_datetime(df_content[col])

    return df_artists, df_content


# ----------------------
# Main Analysis
# ----------------------
def perform_analysis(df_artists, df_content):

    print("⚙️ Running analysis...")

    # ---------- YOUTUBE ENGAGEMENT ----------
    view_col = first_existing_column(df_content, ['view_count', 'yt_view_count'])
    like_col = first_existing_column(df_content, ['like_count'])
    comment_col = first_existing_column(df_content, ['comment_count'])

    df_content['views'] = pd.to_numeric(df_content[view_col], errors='coerce').fillna(0)
    df_content['likes'] = pd.to_numeric(df_content[like_col], errors='coerce').fillna(0)
    df_content['comments'] = pd.to_numeric(df_content[comment_col], errors='coerce').fillna(0)

    df_content['engagement_rate'] = df_content.apply(
        lambda r: (r['likes'] + r['comments']) / r['views'] if r['views'] > 0 else 0,
        axis=1
    )

    # Avg engagement per artist
    avg_eng = df_content.groupby('artist_name')['engagement_rate'].mean().reset_index()
    avg_eng.rename(columns={'engagement_rate': 'avg_yt_engagement'}, inplace=True)

    df_artists = df_artists.merge(avg_eng, on='artist_name', how='left')
    df_artists['avg_yt_engagement'] = df_artists['avg_yt_engagement'].fillna(0)


    # ---------- NUMERIC FIX for artist-level stats ----------
    yt_sub_col = first_existing_column(df_artists, ['yt_subscribers', 'yt_subscribers_numeric'])
    yt_view_col = first_existing_column(df_artists, ['yt_total_views', 'yt_total_views_numeric'])

    tik_views_col = first_existing_column(df_artists, ['tiktok_views_numeric', 'tiktok_views'])
    tik_vids_col = first_existing_column(df_artists, ['tiktok_video_count_numeric', 'tiktok_video_count'])

    df_artists['yt_subscribers'] = pd.to_numeric(df_artists[yt_sub_col], errors='coerce').fillna(0)
    df_artists['yt_total_views'] = pd.to_numeric(df_artists[yt_view_col], errors='coerce').fillna(0)

    df_artists['tiktok_views'] = pd.to_numeric(df_artists[tik_views_col], errors='coerce').fillna(0)
    df_artists['tiktok_video_count'] = pd.to_numeric(df_artists[tik_vids_col], errors='coerce').fillna(0)


    # ---------- NORMALIZATION (manual) ----------
    df_artists['norm_yt_engagement'] = manual_minmax(df_artists['avg_yt_engagement'])
    df_artists['norm_yt_views'] = manual_minmax(df_artists['yt_total_views'])
    df_artists['norm_yt_subs'] = manual_minmax(df_artists['yt_subscribers'])

    df_artists['norm_tiktok_views'] = manual_minmax(df_artists['tiktok_views'])
    df_artists['norm_tiktok_posts'] = manual_minmax(df_artists['tiktok_video_count'])


    # ---------- COMBINED POPULARITY INDEX ----------
    df_artists['combined_popularity_index'] = (
        df_artists['norm_yt_engagement'] * 0.35 +
        df_artists['norm_yt_views'] * 0.20 +
        df_artists['norm_yt_subs'] * 0.15 +
        df_artists['norm_tiktok_views'] * 0.20 +
        df_artists['norm_tiktok_posts'] * 0.10
    )


    # ---------- SAVE RESULTS ----------
    output_path = os.path.join(PROCESSED_DATA_DIR, "analyzed_artists_data.csv")
    df_artists.to_csv(output_path, index=False)
    print(f"✅ Saved analysis → {output_path}")

    print("\n🔥 Top Artists by Popularity:")
    print(df_artists[['artist_name', 'combined_popularity_index']].sort_values(
        'combined_popularity_index', ascending=False
    ).head(10))

    return df_artists


# ----------------------
# Run script
# ----------------------
if __name__ == "__main__":
    df_artists, df_content = load_cleaned_data()
    if df_artists.empty or df_content.empty:
        print("❌ No data available.")
    else:
        perform_analysis(df_artists, df_content)


ModuleNotFoundError: No module named 'sklearn'

In [26]:
# src/visualize_results.py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# Handle paths (works in Jupyter and VS Code)
BASE_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, '../data/processed')
RESULTS_DIR = os.path.join(BASE_DIR, '../results')
os.makedirs(RESULTS_DIR, exist_ok=True)

ANALYZED_ARTISTS_FP = os.path.join(PROCESSED_DATA_DIR, 'analyzed_artists_data.csv')
ANALYZED_YT_AGE_FP = os.path.join(PROCESSED_DATA_DIR, 'analyzed_yt_age_performance.csv')


def load_analyzed_data():
    df_artists = pd.read_csv(ANALYZED_ARTISTS_FP) if os.path.exists(ANALYZED_ARTISTS_FP) else pd.DataFrame()
    df_yt_age = pd.read_csv(ANALYZED_YT_AGE_FP) if os.path.exists(ANALYZED_YT_AGE_FP) else pd.DataFrame()
    return df_artists, df_yt_age


def save_fig(name):
    path = os.path.join(RESULTS_DIR, name)
    plt.savefig(path, bbox_inches='tight')
    print("Saved:", name)


def create_visualizations(df_artists, df_yt_age):
    print("Generating visualizations...")
    sns.set_theme(style="whitegrid")

    # ---------------------------------------------------------
    # 1) Combined Popularity Index Bar Chart
    # ---------------------------------------------------------
    if not df_artists.empty and "combined_popularity_index" in df_artists.columns:
        plt.figure(figsize=(12, 7))
        df_sorted = df_artists.sort_values("combined_popularity_index", ascending=False)
        sns.barplot(
            data=df_sorted,
            x="artist_name",
            y="combined_popularity_index",
            palette="viridis"
        )
        plt.title("Combined Popularity Index by Artist", fontsize=16)
        plt.xlabel("Artist")
        plt.ylabel("Combined Index")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        save_fig("combined_popularity_index.png")
        plt.close()

    # ---------------------------------------------------------
    # 2) YouTube Avg Views by Age Category
    # ---------------------------------------------------------
    if not df_yt_age.empty:
        # ensure category ordering
        age_order = ["0–1 Month", "1–3 Months", "3–12 Months", "1–3 Years", "3+ Years"]
        if "age_category" in df_yt_age.columns:
            df_yt_age["age_category"] = pd.Categorical(df_yt_age["age_category"], age_order, ordered=True)

        plt.figure(figsize=(14, 7))
        sns.barplot(
            data=df_yt_age,
            x="age_category",
            y="avg_views",
            hue="artist_name",
            palette="deep"
        )
        plt.title("Average YouTube Views by Age Category", fontsize=16)
        plt.xlabel("Age Category")
        plt.ylabel("Avg Views")
        plt.xticks(rotation=35)
        plt.legend(title="Artist", bbox_to_anchor=(1.05, 1), loc="upper left")
        plt.tight_layout()
        save_fig("yt_avg_views_by_age.png")
        plt.close()

    # ---------------------------------------------------------
    # 3) Release Frequency vs Popularity
    # ---------------------------------------------------------
    if (
        not df_artists.empty
        and "combined_popularity_index" in df_artists.columns
        and "avg_yt_videos_per_month" in df_artists.columns
    ):
        plt.figure(figsize=(10, 7))
        sns.scatterplot(
            data=df_artists,
            x="avg_yt_videos_per_month",
            y="combined_popularity_index",
            hue="artist_name",
            s=160,
            palette="tab10"
        )
        plt.title("Upload Frequency vs Popularity", fontsize=16)
        plt.xlabel("Avg YouTube Videos per Month")
        plt.ylabel("Popularity Index")
        plt.tight_layout()
        save_fig("release_freq_vs_popularity.png")
        plt.close()

    # ---------------------------------------------------------
    # 4) TikTok Views vs YouTube Subscribers
    # ---------------------------------------------------------
    required_cols = ["tiktok_views", "subscriber_count", "artist_name"]
    if all(col in df_artists.columns for col in required_cols):
        plt.figure(figsize=(10, 7))
        sns.scatterplot(
            data=df_artists,
            x="tiktok_views",
            y="subscriber_count",
            hue="artist_name",
            s=160,
            palette="tab10"
        )
        plt.title("TikTok Views vs YouTube Subscribers", fontsize=16)
        plt.xlabel("TikTok Views")
        plt.ylabel("YouTube Subscribers")
        plt.tight_layout()
        save_fig("tiktok_vs_yt_subscribers.png")
        plt.close()

    # ---------------------------------------------------------
    # 5) Sentiment Breakdown (Donut Chart) for Top Artist
    # ---------------------------------------------------------
    if (
        not df_artists.empty
        and "avg_sentiment_positive" in df_artists.columns
    ):
        top = df_artists.sort_values("combined_popularity_index", ascending=False).iloc[0]

        pos = float(top.get("avg_sentiment_positive", 0))
        neg = float(top.get("avg_sentiment_negative", 0)) if "avg_sentiment_negative" in df_artists.columns else 0
        neu = float(top.get("avg_sentiment_neutral", 0)) if "avg_sentiment_neutral" in df_artists.columns else 1 - pos - neg

        vals = [max(pos, 0), max(neg, 0), max(neu, 0)]
        labels = ["Positive", "Negative", "Neutral"]

        plt.figure(figsize=(7, 7))
        plt.pie(vals, labels=labels, autopct="%1.1f%%", startangle=90)
        centre = plt.Circle((0, 0), 0.65, color="white")
        plt.gca().add_artist(centre)
        plt.title(f"YouTube Sentiment for {top['artist_name']}")
        plt.tight_layout()
        save_fig(f"sentiment_{top['artist_name'].lower().replace(' ', '_')}.png")
        plt.close()

    print("Visualizations complete.")


if __name__ == "__main__":
    df_artists, df_yt_age = load_analyzed_data()
    create_visualizations(df_artists, df_yt_age)


Generating visualizations...
Visualizations complete.
